# Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
from sqlalchemy import text

# 1) SQL

## Engine

In [2]:
host = "localhost"           
port = "5432"                
database = "AACT"
user = "postgres"
password = "pgAdmin4321"
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

# Tables

## my_studies

In [3]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_studies;

CREATE TABLE ctgov.my_studies AS
SELECT DISTINCT ON 
    (
studies.nct_id,
studies.brief_title,
studies.official_title,
studies.source,
studies.phase,
studies.number_of_arms,
studies.enrollment
    ) 
studies.nct_id,
studies.brief_title,
studies.official_title,
studies.overall_status,
studies.phase,
studies.number_of_arms,
studies.enrollment

from ctgov.studies
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status in ('COMPLETED', 'WITHDRAWN', 'TERMINATED')

ORDER BY 
studies.nct_id,
studies.brief_title,
studies.official_title,
studies.source,
studies.phase,
studies.number_of_arms,
studies.enrollment;
"""
"""
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

my_studies = pd.read_sql("select * from ctgov.my_studies;", con = engine)  # was not used anywhere a table. Only for investigational reasons
my_studies

,nct_id,brief_title,official_title,overall_status,phase,number_of_arms,enrollment
0,NCT00125528,D-cycloserine in the Management of Chronic Low...,D-Cycloserine in the Management of Chronic Low...,COMPLETED,PHASE2,2.0,41.0
1,NCT00170209,Rifampin Versus Isoniazid for the Treatment of...,A Randomized Trial to Compare Effectiveness of...,COMPLETED,PHASE3,2.0,844.0
2,NCT00183482,Family Cognitive Behavioral Therapy for Preven...,Family Cognitive Behavioral Prevention of Depr...,COMPLETED,NA,2.0,304.0
3,NCT00194714,Vaccine Therapy in Treating Patients With Stag...,Phase I/II Study of Combination Immunotherapy ...,COMPLETED,PHASE1/PHASE2,1.0,22.0
4,NCT00258791,Effects of Pretreatment With Ibuprofen in Post...,Effects of Pretreatment With Ibuprofen in Post...,WITHDRAWN,NA,NaN,0.0
...,...,...,...,...,...,...,...
182231,NCT06875349,Foot Reflexology and Abdominal Massage,The Effects of Foot Reflexology and Abdominal ...,COMPLETED,NA,3.0,90.0
182232,NCT06875557,Mind Matters; Unveiling the Impact of Digital ...,MIND MATTERS; UNVEILING THE IMPACT OF DIGITAL ...,COMPLETED,PHASE1/PHASE2,2.0,376.0
182233,NCT06875661,Treatment of Adolescent Idiopathic Scoliosis w...,Treatment of Adolescent Idiopathic Scoliosis w...,COMPLETED,NA,2.0,34.0
182234,NCT06875674,Selenium Biofortification of Strawberries,Selenium Biofortification of Strawberries and ...,COMPLETED,NA,3.0,44.0


## my_terminations

In [4]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_terminations;

CREATE TABLE ctgov.my_terminations AS
SELECT DISTINCT ON 
    (
studies.nct_id,
studies.why_stopped,
studies.overall_status
    ) 
studies.nct_id,
studies.why_stopped,
studies.overall_status

from ctgov.studies
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status in ('WITHDRAWN', 'TERMINATED')

ORDER BY studies.nct_id;
"""
"""
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()
""" 
# Enrollment 
my_terminations = pd.read_sql("select * from ctgov.my_terminations;" , con = engine)  # was not used. May be used to a seperate termination ipyd file. (As the y outcome = always termination)
my_terminations.loc[my_terminations["why_stopped"].str.contains(
    "enrollemnt|recru|accru|enrol|participant|recruit|patient|inclusion|human|subjects",
      case = False) == True, "why_stopped2"] = "Enrollment"  
# Fund
my_terminations.loc[my_terminations["why_stopped"].str.contains(
    "fund|sponsor|business|company|portfolio|corporate|resources|financ",
     case = False) == True, "why_stopped2"] = "Fund"
# Administration
my_terminations.loc[my_terminations["why_stopped"].str.contains("PI", case = True) == True, "why_stopped2"] = "Administration"
my_terminations.loc[my_terminations["why_stopped"].str.contains(
    "staff|revise|logistic|administrative|management|strateg|internal|organiz|policy|FDA|decide|decision|change", 
    case = False) == True, "why_stopped2"] = "Administration"

my_terminations.loc[my_terminations["why_stopped"].str.contains(
    "recommend|researcher|Researcher|researchers|Researchers|investigator|Investigator|investigators|Investigators", 
    case = True) == True, "why_stopped2"] = "Administration"
# Efficacy
my_terminations.loc[my_terminations["why_stopped"].str.contains(
    "toxic|futility|efficacy|ethical|safety|sufficient|risk",
      case = False) == True, "why_stopped2"] = "Efficacy"
# Covid
my_terminations.loc[my_terminations["why_stopped"].str.contains(
    "covid|pandemic", case = False) == True, "why_stopped2"] = "Covid"

my_terminations

,nct_id,why_stopped,overall_status,why_stopped2
0,NCT00258791,first postponed then cancelled as national dru...,WITHDRAWN,Administration
1,NCT00293735,Lack of funding,WITHDRAWN,Fund
2,NCT00306059,Investigators decision,WITHDRAWN,Administration
3,NCT00307905,Anticipation of inadequate recruitment accordi...,WITHDRAWN,Enrollment
4,NCT00328809,personnel shortage,WITHDRAWN,NaN
...,...,...,...,...
27654,NCT06841393,Lack of recruitment,TERMINATED,Enrollment
27655,NCT06848556,Company Operational Strategy Adjustment,TERMINATED,Administration
27656,NCT06854432,Lack of recruitment,TERMINATED,Enrollment
27657,NCT06860373,Unable to recruit,TERMINATED,Enrollment


## my_conditions

Source : https://meshb.nlm.nih.gov/treeView

In [5]:
category_map = {
    "A": "Anatomy",
    "B": "Organisms",
    "C": "Diseases",
    "D": "Chemicals, Drugs",
    "E": "Analytical, Diagnostic, Therapeutic Techniques, Equipment",
    "F": "Psychiatry, Psychology",
    "G": "Phenomena, Processes",
    "H": "Disciplines, Occupations",
    "I": "Anthropology, Education, Sociology, Social Phenomena",
    "J": "Technology, Industry, Agriculture",
    "K": "Humanities",
    "L": "Information Science",
    "M": "Named Groups",
    "N": "Health Care",
    "V": "Publication Characteristics",
    "Z": "Geographic Locations"
}

my_qualifiers = pd.DataFrame({"Code" : category_map.keys(), "Category" : category_map.values()})
my_qualifiers

,Code,Category
0,A,Anatomy
1,B,Organisms
2,C,Diseases
3,D,"Chemicals, Drugs"
4,E,"Analytical, Diagnostic, Therapeutic Techniques..."
5,F,"Psychiatry, Psychology"
6,G,"Phenomena, Processes"
7,H,"Disciplines, Occupations"
8,I,"Anthropology, Education, Sociology, Social Phe..."
9,J,"Technology, Industry, Agriculture"


In [ ]:
create_table_sql ="""
DROP TABLE IF EXISTS ctgov.my_conditions;

CREATE TABLE ctgov.my_conditions AS
SELECT DISTINCT ON 
    (
    studies.nct_id,
    mesh_headings.qualifier, 
    mesh_headings.heading,
    conditions.name
    )
    studies.nct_id, 
    mesh_headings.qualifier, 
    mesh_headings.heading,
    conditions.name
    
FROM ctgov.studies
LEFT JOIN ctgov.conditions
    ON studies.nct_id = conditions.nct_id
LEFT JOIN ctgov.browse_conditions
    ON studies.nct_id = browse_conditions.nct_id
LEFT JOIN ctgov.mesh_terms
    ON browse_conditions.mesh_term = mesh_terms.mesh_term
LEFT JOIN ctgov.mesh_headings
    ON mesh_terms.qualifier = mesh_headings.qualifier
    
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
	AND study_type = 'INTERVENTIONAL'
  	AND overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
      
ORDER BY 
studies.nct_id, 
mesh_headings.qualifier, 
mesh_headings.heading,
conditions.name ;  
"""
"""
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()
"""
# Notes --> Why Columns were used or not used:
# 	conditions.name --> use for comorbidity --> not many duplicated consition terms. Less null values than other columns!
#   browse_conditions.mesh_term, # not needed --> more categories than main e.g pain, low back pain, myosceletal etc.
  
my_conditions = pd.read_sql("select * from ctgov.my_conditions;", con = engine)

my_conditions["Code"] = my_conditions["qualifier"].str[0] 
my_conditions = my_conditions.merge(my_qualifiers[["Code", "Category"]], on = "Code", how = "left")

my_conditions = my_conditions.groupby("nct_id").agg({
    "qualifier": lambda x: sorted(set(i for i in x if pd.notnull(i))),
    "heading": lambda x: sorted(set(i for i in x if pd.notnull(i))),
    "Category": lambda x: sorted(set(i for i in x if pd.notnull(i))),
    "Code": lambda x: sorted(set(i for i in x if pd.notnull(i))),
    "name": lambda x: sorted(set(i for i in x if pd.notnull(i))),
}).reset_index()

# C23 ('Pathological Conditions, Signs and Symptoms') has been dropped when len(row) > 1 (>1 qualifiers)
# as it mostly corresponds to symptoms of conditioned noted in the row and thus inflates comorbidity

my_conditions

,nct_id,qualifier,heading,Category,Code,name
0,NCT00125528,"[C10, C23, F02, G11]",[Musculoskeletal and Neural Physiological Phen...,"[Diseases, Phenomena, Processes, Psychiatry, P...","[C, F, G]","[Low Back Pain, Pain]"
1,NCT00170209,[C01],[Bacterial Infections and Mycoses],[Diseases],[C],[Latent Tuberculosis Infection]
2,NCT00183482,"[F01, F03]","[Behavior and Behavior Mechanisms, Mental Diso...","[Psychiatry, Psychology]",[F],[Depression]
3,NCT00194714,"[C04, C12, C13, C19]","[Endocrine System Diseases, Female Urogenital ...",[Diseases],[C],"[HER2/Neu Positive, HLA-A2 Positive Cells Pres..."
4,NCT00258791,"[C10, C23, F02, F03, G11]","[Mental Disorders, Musculoskeletal and Neural ...","[Diseases, Phenomena, Processes, Psychiatry, P...","[C, F, G]",[Mental Disorders]
...,...,...,...,...,...,...
182231,NCT06875349,[C23],"[Pathological Conditions, Signs and Symptoms]",[Diseases],[C],[Constipation]
182232,NCT06875557,[],[],[],[],[Psychological Distress]
182233,NCT06875661,[C05],[Musculoskeletal Diseases],[Diseases],[C],[Idiopathic Scoliosis]
182234,NCT06875674,[],[],[],[],"[Nutrition, Healthy]"


## my_covid

In [7]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_covid;

CREATE TABLE ctgov.my_covid AS
SELECT DISTINCT ON 
(
studies.nct_id, 
browse_conditions.mesh_term
)
    studies.nct_id,
    browse_conditions.mesh_term
    
FROM ctgov.studies
LEFT JOIN ctgov.browse_conditions
    ON studies.nct_id = browse_conditions.nct_id
    
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND study_type = 'INTERVENTIONAL'
  AND overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
  AND browse_conditions.mesh_term ILIKE '%covid%'

ORDER BY 
studies.nct_id, 
browse_conditions.mesh_term;
"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

my_covid = pd.read_sql("select * from ctgov.my_covid;", con = engine)
my_covid = my_covid.drop_duplicates(subset = ["nct_id"])
my_covid["mesh_term"] = my_covid["mesh_term"].replace("Post-Acute COVID-19 Syndrome", "COVID-19")
my_covid

,nct_id,mesh_term
0,NCT03183570,COVID-19
1,NCT03376854,COVID-19
2,NCT03554265,COVID-19
4,NCT03891420,COVID-19
5,NCT03907891,COVID-19
...,...,...
2946,NCT06840873,COVID-19
2948,NCT06850376,COVID-19
2949,NCT06857318,COVID-19
2950,NCT06862531,COVID-19


## my_placebo

In [8]:
create_table_sql ="""
DROP TABLE IF EXISTS ctgov.my_placebo;

CREATE TABLE ctgov.my_placebo AS
SELECT DISTINCT ON (
    studies.nct_id,
    design_groups.group_type   
)
    studies.nct_id,
    design_groups.group_type

    FROM ctgov.studies
LEFT JOIN ctgov.design_groups
    ON studies.nct_id = design_groups.nct_id
    
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
ORDER BY 
    studies.nct_id, 
    design_groups.group_type;
"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""


my_placebo = pd.read_sql("select * from ctgov.my_placebo;", con = engine)
my_placebo

,nct_id,group_type
0,NCT00125528,EXPERIMENTAL
1,NCT00125528,PLACEBO_COMPARATOR
2,NCT00170209,ACTIVE_COMPARATOR
3,NCT00183482,ACTIVE_COMPARATOR
4,NCT00183482,EXPERIMENTAL
...,...,...
288233,NCT06875661,EXPERIMENTAL
288234,NCT06875661,NO_INTERVENTION
288235,NCT06875674,ACTIVE_COMPARATOR
288236,NCT06875674,PLACEBO_COMPARATOR


## my_interventions

In [9]:
create_table_sql ="""
DROP TABLE IF EXISTS ctgov.my_interventions;

CREATE TABLE ctgov.my_interventions AS
SELECT DISTINCT ON (
    studies.nct_id,
    browse_interventions.mesh_term      
)
    studies.nct_id,
    browse_interventions.mesh_term

    FROM ctgov.studies
LEFT JOIN ctgov.browse_interventions
    ON studies.nct_id = browse_interventions.nct_id

WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
ORDER BY 
    studies.nct_id, 
    browse_interventions.mesh_term;
"""
#     interventions.name

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""


my_interventions = pd.read_sql("select * from ctgov.my_interventions;", con = engine)
my_interventions

,nct_id,mesh_term
0,NCT00125528,Anti-Bacterial Agents
1,NCT00125528,Anti-Infective Agents
2,NCT00125528,"Anti-Infective Agents, Urinary"
3,NCT00125528,"Antibiotics, Antitubercular"
4,NCT00125528,Antimetabolites
...,...,...
706936,NCT06875674,Physiological Effects of Drugs
706937,NCT06875674,Protective Agents
706938,NCT06875674,Selenium
706939,NCT06875674,Trace Elements


## my_interventions_types

In [10]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_interventions_types;

CREATE TABLE ctgov.my_interventions_types AS
SELECT DISTINCT ON (
    studies.nct_id,
    interventions.intervention_type
)
    studies.nct_id,
    interventions.intervention_type

FROM ctgov.studies

LEFT JOIN ctgov.interventions
    ON studies.nct_id = interventions.nct_id

WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')

ORDER BY 
    studies.nct_id,
    interventions.intervention_type;
"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

my_interventions_types = pd.read_sql("select * from ctgov.my_interventions_types;", con = engine)
my_interventions_types = my_interventions_types.groupby("nct_id")["intervention_type"].agg(lambda x: sorted(set(i for i in x if i is not None))).reset_index()
# my_interventions_types['intervention_type'] = my_interventions_types['intervention_type'].apply(lambda x: list(set(sorted(x))))
my_interventions_types

,nct_id,intervention_type
0,NCT00125528,[DRUG]
1,NCT00170209,[DRUG]
2,NCT00183482,[BEHAVIORAL]
3,NCT00194714,"[BIOLOGICAL, OTHER]"
4,NCT00258791,[DRUG]
...,...,...
182231,NCT06875349,[OTHER]
182232,NCT06875557,[BEHAVIORAL]
182233,NCT06875661,[OTHER]
182234,NCT06875674,[DIETARY_SUPPLEMENT]


## my_intervention_methods / my_intervention_methods2
(?<!\w): no word character before (start of word)
(?!\w): no word character after (end of word)
\s --> Space before/sfter (no character of world/non world) 
\b --> word boundary. (Exception: Non letter characters could exist e.g -.,)
\. --> period as string e.g i.v (while str1.str2 = any character betwwen str1 and str2)
? --> optional
^ --> not 
[^a-zA-Z] --> non world/letter character

In [11]:
# my_intervention_methods2
create_table_sql ="""
DROP TABLE IF EXISTS ctgov.my_intervention_methods2;

CREATE TABLE ctgov.my_intervention_methods2 AS
SELECT DISTINCT ON 
(
    studies.nct_id,
    interventions.description, 
    design_groups.description,
    design_groups.group_type
    
)
    studies.nct_id,
    design_groups.description AS design_groups_description,
    interventions.description AS interventions_description,
    design_groups.group_type

    FROM ctgov.studies

LEFT JOIN ctgov.interventions
    ON studies.nct_id = interventions.nct_id
LEFT JOIN ctgov.design_groups
    ON studies.nct_id = design_groups.nct_id

WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
ORDER BY 
    studies.nct_id,
    design_groups.group_type;
"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

my_intervention_methods3 = pd.read_sql("select * from ctgov.my_intervention_methods2;", con = engine)
my_intervention_methods2 = my_intervention_methods3.copy()
my_intervention_methods2 = my_intervention_methods2.drop(columns = ['design_groups_description', 'group_type'])
my_intervention_methods2 = my_intervention_methods2.drop_duplicates(subset = ['nct_id', 'interventions_description'])

# =======================================================================================================
# my_intervention_methods
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_intervention_methods;

CREATE TABLE ctgov.my_intervention_methods AS

select distinct on 
(
nct_id, 
intervention_type, 
description
) 
studies.nct_id,
interventions.intervention_type,
interventions.description

from ctgov.studies
left join ctgov.interventions
on studies.nct_id = interventions.nct_id

WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
order by studies.nct_id, interventions.description, interventions.intervention_type;
"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

my_intervention_methods = pd.read_sql("select * from ctgov.my_intervention_methods;", con = engine)

display(my_intervention_methods2, my_intervention_methods)

,nct_id,interventions_description
0,NCT00125528,D-cycloserine 50 mg bid; D-cycloserine 100 mg ...
1,NCT00125528,placebo bid
4,NCT00170209,The dosage of the medication is determined acc...
6,NCT00170209,The dosage of the medication is determined acc...
8,NCT00183482,Families are mailed written materials that des...
...,...,...
758212,NCT06875661,Patients in the experimental group exercised f...
758214,NCT06875674,control strawberry supplementation (100 grams/...
758216,NCT06875674,selenium biofortified strawberry supplementati...
758218,NCT06875674,selenium supplementation in tablet (100 microg...


,nct_id,intervention_type,description
0,NCT00125528,DRUG,D-cycloserine 50 mg bid; D-cycloserine 100 mg ...
1,NCT00125528,DRUG,placebo bid
2,NCT00170209,DRUG,The dosage of the medication is determined acc...
3,NCT00170209,DRUG,The dosage of the medication is determined acc...
4,NCT00183482,BEHAVIORAL,Families are mailed written materials that des...
...,...,...,...
327435,NCT06875661,OTHER,Patients in the experimental group exercised f...
327436,NCT06875674,DIETARY_SUPPLEMENT,control strawberry supplementation (100 grams/...
327437,NCT06875674,DIETARY_SUPPLEMENT,selenium biofortified strawberry supplementati...
327438,NCT06875674,DIETARY_SUPPLEMENT,selenium supplementation in tablet (100 microg...


In [12]:
# Merge all different descriptions per nct_id (intervention_type) 
my_intervention_methods['description'] = my_intervention_methods.groupby('nct_id')['description'].transform(
    lambda x: ['|'.join(i for i in x if pd.notnull(i))] * len(x)) # same number of rows = 327k
    # SOS tranform wont drop intervention_type column --> no need to groupby that also

my_intervention_methods2['interventions_description'] = my_intervention_methods2.groupby('nct_id')\
    ['interventions_description'].transform(
    lambda x: ['|'.join(i for i in x if pd.notnull(i))] * len(x))

# Drop Duplicates
my_intervention_methods = my_intervention_methods.drop_duplicates(subset=['nct_id', 'intervention_type', 'description'])
my_intervention_methods2 = my_intervention_methods2.drop_duplicates(subset = ['nct_id', 'interventions_description'])

# display(my_intervention_methods2, my_intervention_methods)

# ==============================================================================
# Merge both dfs
my_intervention_methods = my_intervention_methods.merge(my_intervention_methods2[['nct_id', 'interventions_description']],
                            on = "nct_id", how = "outer") 

my_intervention_methods['interv_desc'] = my_intervention_methods[['description', 'interventions_description']].apply(
    lambda x: '|'.join(str(i) for i in x if pd.notnull(i)), axis = 1)

# Drop intial columns
my_intervention_methods = my_intervention_methods.drop(columns = ['description', 'interventions_description'], axis = 1)

# ==============================================================================
# Split to list
my_intervention_methods['interv_desc'] = my_intervention_methods['interv_desc'].apply(lambda x: x.split('|') if x != '|' else [])
# Unique list values
my_intervention_methods['interv_desc'] =  my_intervention_methods['interv_desc'].apply(
  lambda x: sorted(set(x))  if isinstance(x, list) else x)

# ==============================================================================
# Remake string to drop duplicates
my_intervention_methods['interv_desc'] = my_intervention_methods['interv_desc'].apply(
    lambda x: ' | '.join(x) if isinstance(x, list) else str(x))

# Drop Dusplicated rows
my_intervention_methods = my_intervention_methods.drop_duplicates(subset = ['nct_id', 'interv_desc', 'intervention_type'])

display(my_intervention_methods['nct_id'].nunique()) # same nunique as started --> not nct_ids lost
my_intervention_methods

182236

,nct_id,intervention_type,interv_desc
0,NCT00125528,DRUG,D-cycloserine 50 mg bid; D-cycloserine 100 mg ...
1,NCT00170209,DRUG,The dosage of the medication is determined acc...
2,NCT00183482,BEHAVIORAL,Families are mailed written materials that des...
3,NCT00194714,OTHER,Correlative studies | Given ID
4,NCT00194714,BIOLOGICAL,Correlative studies | Given ID
...,...,...,...
209156,NCT06875349,OTHER,Abdominal massage is a therapeutic technique t...
209157,NCT06875557,BEHAVIORAL,Digital Mental Health Intervention (DMHI) was ...
209158,NCT06875661,OTHER,Patients in the experimental group exercised f...
209159,NCT06875674,DIETARY_SUPPLEMENT,control strawberry supplementation (100 grams/...


In [ ]:
# Injection
injection1 = (
    r"\binject|\bvaccin|\bbolus\b|\bvial\b|"
    r"\barterial|\bepidural|"
    r"\bsyring|\binfuse\b|\binfusion\b|\bimmunization\b|\biv[^a-zA-Z]?push\b|"
    r"\bdrip\b|\bchemotherapy\b|\binsulin\b|\bparenteral\b|\bblood\b|"
    r"\bantibod\b|\bgraft\b|cells/kg|mg/m2|gm/m2|platin\b|zamab.cyrcles\b|" 

    r"\bsub[^a-zA-Z]?(?:cutaneous)|"
    r"\bintra[^a-zA-Z]?(?:arterial|muscular|venous|thecal|osseous|peritoneal|dermal|ocular|urethral|uterine|articular|\
    tympanic|cutaneous|spinal|ventricular|lesional|lymphatic)\b"
)

injection2 = (
    r"(?<!\w)(?:iv|i\.v\.|im|i\.m\.|i\.a\.|sc|sq|it|ip|ivpush|ivdrip|id|\bcc\b|\bc\.c\b)(?!\w)"
)

# Surgical
surgical1 = (
    r"\bsurg|\boperat|\bincision|\bresection|\btransplant|"
    r"\bstent\b|\blaparoscop|\bablation\b|\bimplant|micro[^a-zA-Z]?surgery|"
    r"\btrephinat|\bamputat|\bbiopsy\b|bariatric surgery|"
    r"\bexcision\b|oplasty\b|\bbypass\b|anastomosis|operation|"
    r"\banastomos|\baugmentation\b|\bsutur|lead placement|"
    r"[a-zA-Z]*(?:ectomy|oscopy|otomy)\b"    # oscopy e.g colonoscopy, with the meaning of more interventional examine.
)

# Oral
oral1 = (
    r"\boral|\borally\b|tablet|\btabs\b|\btab\b|capsule|pill|supplement|mouth|sublingual|"
    r"chew|lozenge|syrup|liquid|elixir|granule|\blick|sublingual|buccal"
    )

oral2 = (
    r"\bpo\b|\bper os\b|\bp\.o\.\b|\bp\.o\.\b|\bperos\b"
    )

# Topical
topical1 = (
    r"\bgel\b|gels|patch|patches|cream|creams|ointment|ointments|balm|balms|nebul|" \
    r"paste|eye drop|ear drop|cm2 square|"
    r"nebulizer|toothpaste|intranasal|nasal|inhal|inhaler|aerosol|lotion|lotions|"
    r"vaginal|rectal|spray|sprays|drops|ophthalm|\botic|transdermal|suppository|"
    r"douche|mouthwash|swish|enema|gargle|radio|radiation|photo-therpy|phototherapy|" \
    r"dermabrasion|ultrasound"
)
# Save for using in Thesis file too
interv_data = {
    "oral1": oral1,
    "oral2":oral2,
    "injection1": injection1,
    "injection2": injection2,
    "topical1": topical1,
    "surgical1": surgical1}

with open("intervention_methods.pkl", "wb") as f:
    pickle.dump(interv_data, f)

# ==============================================================================================================================#
# Search intervention worlds within text
# Injection
my_intervention_methods.loc[
    my_intervention_methods['interv_desc'].str.contains(injection1, case = False, na = False), 
    'Injection'] = "Injection"
my_intervention_methods.loc[
    my_intervention_methods['interv_desc'].str.lower().str.contains(injection2, case = True, na=False),
    'Injection'] = "Injection"  # str.lower() --> case = True
# Oral
my_intervention_methods.loc[
    my_intervention_methods['interv_desc'].str.contains(oral1, case = False, na = False), 
    'Oral'] = "Oral"
my_intervention_methods.loc[
    my_intervention_methods['interv_desc'].str.lower().str.contains(oral2.lower(), case = True, na = False), 
    'Oral'] = "Oral"  # str.lower() --> case = True
# Topical
my_intervention_methods.loc[
    my_intervention_methods['interv_desc'].str.contains(topical1, case = False, na = False), 
    'Topical'] = "Topical"
# Surgery
my_intervention_methods.loc[
    my_intervention_methods['interv_desc'].str.contains(surgical1, case = False, na = False),
    'Surgical'] = "Surgical"

# ==============================================================================================================================#
# Merge all columns to 1
my_intervention_methods["interv_method"] = my_intervention_methods[["Injection", "Oral", "Topical", "Surgical"]].apply(
    lambda row: [val for val in row if pd.notna(val)], axis = 1)

my_intervention_methods = my_intervention_methods.drop(columns = ['Oral', 'Topical', 'Injection', 'Surgical'])

# ==============================================================================================================================#
# Fill BEHAVIORAL nan rows
my_intervention_methods.loc[my_intervention_methods['intervention_type'].apply(lambda x: x == 'BEHAVIORAL') &
                            my_intervention_methods['interv_method'].apply(lambda x: x ==[])] = 'non_interv'

# ==============================================================================================================================#
# Check 'interv_desc' texts and 'interv_method' outputs
my_intervention_methods  


,nct_id,intervention_type,interv_desc,interv_method
0,NCT00125528,DRUG,D-cycloserine 50 mg bid; D-cycloserine 100 mg ...,[]
1,NCT00170209,DRUG,The dosage of the medication is determined acc...,[Oral]
2,non_interv,non_interv,non_interv,non_interv
3,NCT00194714,OTHER,Correlative studies | Given ID,[Injection]
4,NCT00194714,BIOLOGICAL,Correlative studies | Given ID,[Injection]
...,...,...,...,...
209156,NCT06875349,OTHER,Abdominal massage is a therapeutic technique t...,[Injection]
209157,NCT06875557,BEHAVIORAL,Digital Mental Health Intervention (DMHI) was ...,[Injection]
209158,NCT06875661,OTHER,Patients in the experimental group exercised f...,[]
209159,NCT06875674,DIETARY_SUPPLEMENT,control strawberry supplementation (100 grams/...,[Oral]


In [14]:
# Drop interv_desc text columns after checking
my_intervention_methods = my_intervention_methods.drop(columns = ['interv_desc'])

# groupby nct_id --> intervenion_type column dropped
my_intervention_methods = my_intervention_methods.groupby("nct_id")["interv_method"].agg(
    lambda col: sorted(set(elem for row in col if isinstance(row, list) for elem in row))
    ).reset_index()

#  Display np.nan
my_intervention_methods['interv_method'] = my_intervention_methods['interv_method'].apply(lambda x: np.nan if x == [] else x)
my_intervention_methods = my_intervention_methods.dropna()
my_intervention_methods


,nct_id,interv_method
1,NCT00170209,[Oral]
2,NCT00194714,[Injection]
4,NCT00263432,[Surgical]
5,NCT00293735,"[Injection, Oral]"
10,NCT00329641,"[Injection, Oral]"
...,...,...
164685,NCT06874777,"[Injection, Surgical, Topical]"
164688,NCT06875050,[Oral]
164689,NCT06875349,[Injection]
164690,NCT06875557,[Injection]


## my_soc

In [61]:
my_intervention_methods3

,nct_id,design_groups_description,interventions_description,group_type
0,NCT00125528,D-cycloserine 50mg bid/100mg bid/200 mg bid,D-cycloserine 50 mg bid; D-cycloserine 100 mg ...,EXPERIMENTAL
1,NCT00125528,D-cycloserine 50mg bid/100mg bid/200 mg bid,placebo bid,EXPERIMENTAL
2,NCT00125528,placebo,D-cycloserine 50 mg bid; D-cycloserine 100 mg ...,PLACEBO_COMPARATOR
3,NCT00125528,placebo,placebo bid,PLACEBO_COMPARATOR
4,NCT00170209,The experimental arm will be daily self-admini...,The dosage of the medication is determined acc...,ACTIVE_COMPARATOR
...,...,...,...,...
758219,NCT06875674,This group will received selenium in tablet (1...,selenium supplementation in tablet (100 microg...,ACTIVE_COMPARATOR
758220,NCT06875674,The control group received strawberries (contr...,control strawberry supplementation (100 grams/...,PLACEBO_COMPARATOR
758221,NCT06875674,The control group received strawberries (contr...,selenium biofortified strawberry supplementati...,PLACEBO_COMPARATOR
758222,NCT06875674,The control group received strawberries (contr...,selenium supplementation in tablet (100 microg...,PLACEBO_COMPARATOR


In [62]:
my_soc = my_intervention_methods3.copy()

pattern = r"(?:standard|usual)\s*(?:of\s*)?(?:therapy|care|treatment|CPR)\b"

# Apply condition to create the new column
my_soc['Standard_Care'] = (
    my_soc["design_groups_description"].str.contains(pattern, case=False, na=False) |
    my_soc["interventions_description"].str.contains(pattern, case=False, na=False)
    ).map({True: "Yes", False: np.nan})

my_soc = my_soc[['nct_id', 'Standard_Care']].dropna()
my_soc = my_soc.drop_duplicates(subset = ["nct_id", 'Standard_Care']).reset_index(drop = True)
my_soc

,nct_id,Standard_Care
0,NCT00170209,Yes
1,NCT00459641,Yes
2,NCT00700154,Yes
3,NCT00795028,Yes
4,NCT00826059,Yes
...,...,...
10852,NCT06863922,Yes
10853,NCT06864429,Yes
10854,NCT06867939,Yes
10855,NCT06868420,Yes


## my_adverse

### my_adverse

In [63]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_adverse;

CREATE TABLE ctgov.my_adverse AS
SELECT DISTINCT ON 
(
  studies.nct_id, 
	reported_event_totals.event_type,
	reported_event_totals.subjects_affected
  )
  studies.nct_id, 
	reported_event_totals.event_type,
	reported_event_totals.subjects_affected as event_count
	
FROM ctgov.studies
LEFT JOIN ctgov.reported_event_totals
  ON studies.nct_id = reported_event_totals.nct_id

WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')

ORDER BY 
	studies.nct_id, 
    reported_event_totals.event_type;
"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

# drop_withdrawals.reason as withrawals_reason,
# drop_withdrawals.count as withrawals_count,
#	reported_event_totals.classification,
#	reported_event_totals.classification as event_classification,

# LEFT JOIN ctgov.drop_withdrawals
# ON studies.nct_id = drop_withdrawals.nct_id

my_adverse = pd.read_sql("select * from ctgov.my_adverse;", con = engine)
my_adverse = my_adverse.replace({'other':'Other', 'serious':'Serious', 'deaths': 'Death'})  # Do not replace [] --> event_count is needed as 0
my_adverse.loc[my_adverse["event_count"] == 0.0, ["event_type", "event_count"]] = np.nan    # nan = []
my_adverse = my_adverse.groupby("nct_id").agg({
    "event_type": lambda x: sorted(set(i for i in x if pd.notnull(i))),
    "event_count": lambda x: sorted(set(i for i in x if pd.notnull(i))),
}).reset_index()
my_adverse["event_sum"] = my_adverse["event_count"].apply(lambda x: sum(i for i in x if pd.notnull(i)))
my_adverse

,nct_id,event_type,event_count,event_sum
0,NCT00125528,"[Death, Other]","[3.0, 5.0]",8.0
1,NCT00170209,[],[],0.0
2,NCT00183482,[],[],0.0
3,NCT00194714,"[Death, Other, Serious]","[3.0, 11.0, 21.0]",35.0
4,NCT00258791,[],[],0.0
...,...,...,...,...
182231,NCT06875349,[],[],0.0
182232,NCT06875557,[],[],0.0
182233,NCT06875661,[],[],0.0
182234,NCT06875674,[],[],0.0


### my_adverse_system

In [64]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_adverse_system;

CREATE TABLE ctgov.my_adverse_system AS
SELECT DISTINCT ON
(
studies.nct_id,
reported_events.organ_system
)
studies.nct_id,
reported_events.organ_system

FROM ctgov.reported_events
LEFT JOIN ctgov.studies
    ON studies.nct_id = reported_events.nct_id

WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status in ('COMPLETED','WITHDRAWN', 'TERMINATED')
ORDER BY studies.nct_id;
"""

with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()

my_adverse_system = pd.read_sql("select * from ctgov.my_adverse_system;", con = engine)

# transformed to more general categories, as samples where to small per category as seen in thesis_vizual.ipyd
my_adverse_system['organ_system'] = my_adverse_system['organ_system'].replace({
    
      'General disorders' : 'General',
      'Infections and infestations' : 'Infections',
      'Immune system disorders' : 'Immune System',

      'Nervous system disorders' : 'Nervous System',
      'Psychiatric disorders' : 'Psychiatric/ Social', 
      'Social circumstances' : 'Social',

      'Skin and subcutaneous tissue disorders' : 'Skin',
      'Injury, poisoning and procedural complications' : 'Injury/ Poisoning/ Procedural',
      'Surgical and medical procedures' : 'Surgical/ Medical', 

      'Blood and lymphatic system disorders' : 'Blood/ Lymphatic', 
      'Hepatobiliary disorders' : 'Hepatobiliary',  
      'Endocrine disorders' : 'Endocrine',    

      'Cardiac disorders' : 'Cardio',
      'Vascular disorders': 'Vascular',

      'Renal and urinary disorders' : 'Renal/ Urinary',
      
      'Respiratory, thoracic and mediastinal disorders' : 'Respiratory',

      'Gastrointestinal disorders' : 'Gastrointestinal',
      'Metabolism and nutrition disorders' : 'Metabolism/ Nutrition',

      'Musculoskeletal and connective tissue disorders' : 'Musculoskeletal',

      'Ear and labyrinth disorders' : 'Ear', 
      'Eye disorders' : 'Eye', 

      'Neoplasms benign, malignant and unspecified (incl cysts and polyps)' : 'Neoplasms',

      'Congenital, familial and genetic disorders' : 'Genetic',
      'Reproductive system and breast disorders' : 'Reproductive/ Breast',
      'Pregnancy, puerperium and perinatal conditions' : 'Pregnancy/ Perinatal'
      # 'Investigations',
      # 'Product Issues', 
      })

my_adverse_system["organ_system"] = my_adverse_system["organ_system"].replace(r"\s*disorders$", "", regex=True)
my_adverse_system = my_adverse_system.groupby("nct_id").agg({"organ_system": lambda x: sorted(set(i for i in x if pd.notnull(i)))}).reset_index()
my_adverse_system

,nct_id,organ_system
0,NCT00125528,"[General, Nervous System, Skin]"
1,NCT00194714,"[Blood/ Lymphatic, Cardio, Eye, Gastrointestin..."
2,NCT00329641,"[Blood/ Lymphatic, Eye, Gastrointestinal, Gene..."
3,NCT00397579,"[Blood/ Lymphatic, Hepatobiliary]"
4,NCT00422422,"[Gastrointestinal, General, Infections, Invest..."
...,...,...
30768,NCT06469138,"[Gastrointestinal, General, Infections, Nervou..."
30769,NCT06582732,"[Cardio, Eye, Infections, Injury/ Poisoning/ P..."
30770,NCT06610279,"[Cardio, Gastrointestinal, General, Infections..."
30771,NCT06655818,"[Blood/ Lymphatic, Eye, Gastrointestinal, Gene..."


## my_designs

In [65]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_designs;

CREATE TABLE ctgov.my_designs AS
SELECT DISTINCT ON 
(
studies.nct_id,
designs.allocation,
designs.intervention_model,
designs.primary_purpose,
designs.masking,
designs.subject_masked,
designs.caregiver_masked,
designs.investigator_masked,
designs.outcomes_assessor_masked
)

studies.nct_id,
designs.allocation,
designs.intervention_model,
designs.primary_purpose,
designs.masking,
designs.subject_masked,
designs.caregiver_masked,
designs.investigator_masked,
designs.outcomes_assessor_masked

FROM ctgov.studies
LEFT JOIN ctgov.designs
  ON studies.nct_id = designs.nct_id
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
ORDER BY studies.nct_id;

"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

my_designs = pd.read_sql("select * from ctgov.my_designs;", con = engine)

my_designs["subject_masked"] = my_designs["subject_masked"].replace({True : "PARTICIPANT", False : np.NaN})
my_designs["caregiver_masked"] = my_designs["caregiver_masked"].replace({True : "CARE_PROVIDER", False : np.NaN})
my_designs["investigator_masked"] = my_designs["investigator_masked"].replace({True : "INVESTIGATOR", False : np.NaN})
my_designs["outcomes_assessor_masked"] = my_designs["outcomes_assessor_masked"].replace({True : "OUTCOMES_ASSESSOR", False : np.NaN})

my_designs["masking_detail"] = my_designs[["subject_masked", "caregiver_masked", "investigator_masked", "outcomes_assessor_masked"]].apply(
    lambda row: [val for val in row if pd.notna(val) and val != "None"], axis=1)
my_designs['masking_detail'] = my_designs['masking_detail'].apply(lambda x: list(set(sorted(x))))
my_designs

,nct_id,allocation,intervention_model,primary_purpose,masking,subject_masked,caregiver_masked,investigator_masked,outcomes_assessor_masked,masking_detail
0,NCT00125528,RANDOMIZED,PARALLEL,TREATMENT,TRIPLE,PARTICIPANT,NaN,INVESTIGATOR,OUTCOMES_ASSESSOR,"[OUTCOMES_ASSESSOR, PARTICIPANT, INVESTIGATOR]"
1,NCT00170209,RANDOMIZED,PARALLEL,TREATMENT,NONE,None,None,None,None,[]
2,NCT00183482,RANDOMIZED,PARALLEL,PREVENTION,DOUBLE,NaN,NaN,INVESTIGATOR,OUTCOMES_ASSESSOR,"[OUTCOMES_ASSESSOR, INVESTIGATOR]"
3,NCT00194714,NA,SINGLE_GROUP,TREATMENT,NONE,None,None,None,None,[]
4,NCT00258791,RANDOMIZED,CROSSOVER,TREATMENT,DOUBLE,PARTICIPANT,NaN,NaN,OUTCOMES_ASSESSOR,"[PARTICIPANT, OUTCOMES_ASSESSOR]"
...,...,...,...,...,...,...,...,...,...,...
182231,NCT06875349,RANDOMIZED,CROSSOVER,TREATMENT,NONE,None,None,None,None,[]
182232,NCT06875557,NA,PARALLEL,HEALTH_SERVICES_RESEARCH,DOUBLE,PARTICIPANT,NaN,INVESTIGATOR,NaN,"[PARTICIPANT, INVESTIGATOR]"
182233,NCT06875661,RANDOMIZED,PARALLEL,TREATMENT,NONE,None,None,None,None,[]
182234,NCT06875674,RANDOMIZED,PARALLEL,PREVENTION,TRIPLE,PARTICIPANT,NaN,INVESTIGATOR,OUTCOMES_ASSESSOR,"[OUTCOMES_ASSESSOR, PARTICIPANT, INVESTIGATOR]"


## my_eligibilities

In [66]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_eligibilities;

CREATE TABLE ctgov.my_eligibilities AS
SELECT DISTINCT ON 
(
studies.nct_id, 
eligibilities.gender, 
eligibilities.healthy_volunteers
) 
studies.nct_id, 
eligibilities.gender, 
eligibilities.healthy_volunteers
                               
FROM ctgov.studies
LEFT JOIN ctgov.eligibilities
	ON studies.nct_id = eligibilities.nct_id
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status in ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
ORDER BY studies.nct_id;
"""
# eligibilities.minimum_age, 
# eligibilities.maximum_age, 

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

    
my_eligibilities = pd.read_sql("select * from ctgov.my_eligibilities;", con = engine)
my_eligibilities["healthy_volunteers"] = my_eligibilities["healthy_volunteers"].replace({True : "Healthy", False : "Condition", None : np.NaN})
my_eligibilities

,nct_id,gender,healthy_volunteers
0,NCT00125528,ALL,Condition
1,NCT00170209,ALL,Condition
2,NCT00183482,ALL,Condition
3,NCT00194714,FEMALE,Condition
4,NCT00258791,ALL,Condition
...,...,...,...
182231,NCT06875349,ALL,Condition
182232,NCT06875557,ALL,Condition
182233,NCT06875661,ALL,Condition
182234,NCT06875674,ALL,Healthy


## my_outcomes

In [67]:
create_table_sql ="""
DROP TABLE IF EXISTS ctgov.my_outcomes;

CREATE TABLE ctgov.my_outcomes AS
SELECT DISTINCT ON 
( 
    studies.nct_id, 
    outcomes.outcome_type 
)
    studies.nct_id, 
    outcomes.outcome_type 

FROM ctgov.studies

LEFT JOIN ctgov.outcomes
  ON studies.nct_id = outcomes.nct_id

WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status in ('COMPLETED', 'WITHDRAWN', 'TERMINATED')

ORDER BY 
    studies.nct_id, 
    outcomes.outcome_type;

"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""
   
my_outcomes = pd.read_sql("select * from ctgov.my_outcomes;", con = engine)
my_outcomes = my_outcomes.groupby("nct_id")["outcome_type"].agg(lambda x: sorted(set(i for i in x if i is not None))).reset_index()
my_outcomes

,nct_id,outcome_type
0,NCT00125528,"[PRIMARY, SECONDARY]"
1,NCT00170209,[]
2,NCT00183482,[]
3,NCT00194714,"[PRIMARY, SECONDARY]"
4,NCT00258791,[]
...,...,...
182231,NCT06875349,[]
182232,NCT06875557,[]
182233,NCT06875661,[]
182234,NCT06875674,[]


## my_locations

In [68]:
create_table_sql ="""
DROP TABLE IF EXISTS ctgov.my_locations;

CREATE TABLE ctgov.my_locations AS
SELECT DISTINCT ON (
    studies.nct_id,
    countries.name,
    facilities.country,
    facilities.city
)
    studies.nct_id,
    countries.name AS countries_country,
    facilities.country AS facilities_country,
    facilities.city AS facilities_city
    
FROM ctgov.studies 
LEFT JOIN ctgov.facilities
    ON studies.nct_id = facilities.nct_id
LEFT JOIN ctgov.countries
    ON studies.nct_id = countries.nct_id
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status IN ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
ORDER BY 
    studies.nct_id, 
    countries.name, 
    facilities.country, 
    facilities.city;

"""
# facilities.state, 
"""
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()
"""
 
my_locations = pd.read_sql("select * from ctgov.my_locations;", con = engine)

# Fix Country names based on geoDict used in Thesis_Data File
country_map = {
    'Korea, Republic of': 'South Korea',
    'Iran, Islamic Republic of': 'Iran',
    'Syrian Arab Republic': 'Syria',
    'Russian Federation': 'Russia',
    'Czech Republic': 'Czechia',
    'Moldova, Republic of': 'Moldova',
    'Palestinian Territory, occupied': 'Palestinian Territory',
    'Palestinian Territories, Occupied': 'Palestinian Territory',
    "Lao People's Democratic Republic": 'Laos',
    'Libyan Arab Jamahiriya': 'Libya',
    'Macedonia, The Former Yugoslav Republic of': 'North Macedonia',
    'Brunei Darussalam': 'Brunei',
    'Federated States of Micronesia': 'Micronesia',
    'Réunion': 'Reunion',
    'Congo, The Democratic Republic of the': 'Republic of the Congo',
    'Congo': 'Republic of the Congo',
    'Timor-Leste': 'Timor Leste'
}
# countries_country
my_locations['countries_country'] = my_locations['countries_country'].apply(
    lambda x: [country_map.get(x[0], x[0])] if isinstance(x, list) and len(x) == 1 else x)
# facilities_country
my_locations['facilities_country'] = my_locations['facilities_country'].apply(
    lambda x: [country_map.get(x[0], x[0])] if isinstance(x, list) and len(x) == 1 else x)

my_locations = my_locations.groupby("nct_id").agg({
    "countries_country": lambda x: sorted(set(i for i in x if pd.notnull(i))),
    "facilities_country": lambda x: sorted(set(i for i in x if pd.notnull(i))),
    "facilities_city": lambda x: sorted(set(i for i in x if pd.notnull(i)))
}).reset_index()

# Fix missing facilities County --> Most cases both columns are the same
constr = (my_locations['countries_country'] != my_locations['facilities_country']) &\
      (my_locations['facilities_country'].apply(lambda x: len(x) == 0))
my_locations.loc[constr, 'facilities_country'] = my_locations.loc[constr, 'countries_country']

my_locations["Country_Counts"] = my_locations["facilities_country"].apply(lambda x: len(x))
my_locations["City_Counts"] = my_locations["facilities_city"].apply(lambda x: len(x))
my_locations

,nct_id,countries_country,facilities_country,facilities_city,Country_Counts,City_Counts
0,NCT00125528,[United States],[United States],[Chicago],1,1
1,NCT00170209,"[Australia, Benin, Brazil, Canada, Ghana, Guin...","[Australia, Benin, Brazil, Canada, Ghana, Guin...","[Bandung, Conakry, Cotonou, Edmonton, Kumasi, ...",7,9
2,NCT00183482,[United States],[United States],[Nashville],1,1
3,NCT00194714,[United States],[United States],[Seattle],1,1
4,NCT00258791,[Norway],[Norway],[],1,0
...,...,...,...,...,...,...
182231,NCT06875349,[Turkey],[Turkey],[Trabzon],1,1
182232,NCT06875557,[Pakistan],[Pakistan],[Rawalpindi],1,1
182233,NCT06875661,[Serbia],[Serbia],[Novi Sad],1,1
182234,NCT06875674,[Italy],[Italy],[Palermo],1,1


## my_documents

In [69]:
create_table_sql ="""
DROP TABLE IF EXISTS ctgov.my_documents;

CREATE TABLE ctgov.my_documents AS
SELECT DISTINCT ON 
( 
studies.nct_id, 
provided_documents.has_protocol, 
provided_documents.has_icf, 
provided_documents.has_sap
)
studies.nct_id, 
provided_documents.has_protocol, 
provided_documents.has_icf, 
provided_documents.has_sap 
FROM ctgov.studies
LEFT JOIN ctgov.provided_documents
  ON studies.nct_id = provided_documents.nct_id
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status in ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
ORDER BY studies.nct_id;

"""

"""with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""
    
my_documents = pd.read_sql("select * from ctgov.my_documents;", con = engine)

my_documents["has_protocol"] = my_documents["has_protocol"].replace({True : "Protocol", False : np.NaN})
my_documents["has_icf"] = my_documents["has_icf"].replace({True : "Consent Form", False : np.NaN})
my_documents["has_sap"] = my_documents["has_sap"].replace({True : "Analysis Plan", False : np.NaN})

my_documents = my_documents.groupby("nct_id", as_index=False).agg({
    'has_protocol': 'first',
    'has_icf': 'first',
    'has_sap': 'first'
}).reset_index()

my_documents["Study_Documents"] = my_documents[["has_protocol", "has_icf", "has_sap"]].apply(
    lambda row: [val for val in row if pd.notna(val) and val != "None"],
    axis=1)
my_documents['Study_Documents'] = my_documents['Study_Documents'].apply(lambda x: list(set(sorted(x))))
my_documents

,index,nct_id,has_protocol,has_icf,has_sap,Study_Documents
0,0,NCT00125528,None,None,None,[]
1,1,NCT00170209,None,None,None,[]
2,2,NCT00183482,None,None,None,[]
3,3,NCT00194714,Protocol,Consent Form,Analysis Plan,"[Protocol, Consent Form, Analysis Plan]"
4,4,NCT00258791,None,None,None,[]
...,...,...,...,...,...,...
182231,182231,NCT06875349,None,None,None,[]
182232,182232,NCT06875557,None,None,None,[]
182233,182233,NCT06875661,None,None,None,[]
182234,182234,NCT06875674,None,None,None,[]


## my_source

In [70]:
create_table_sql = """
DROP TABLE IF EXISTS ctgov.my_source;

CREATE TABLE ctgov.my_source AS
select 
distinct on 
(
studies.nct_id,
studies.source,
studies.source_class
)
studies.nct_id,
studies.source,
studies.source_class
from ctgov.studies
WHERE TO_CHAR(studies.start_date, 'YYYY-MM') >= '2011-01'
  AND studies.study_type = 'INTERVENTIONAL'
  AND studies.overall_status in ('COMPLETED', 'WITHDRAWN', 'TERMINATED')
order by 
studies.nct_id,
studies.source,
studies.source_class;
"""
"""
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()"""

my_source = pd.read_sql("select * from ctgov.my_source;", con = engine)
my_source

,nct_id,source,source_class
0,NCT00125528,Northwestern University,OTHER
1,NCT00170209,McGill University,OTHER
2,NCT00183482,Vanderbilt University,OTHER
3,NCT00194714,University of Washington,OTHER
4,NCT00258791,Norwegian University of Science and Technology,OTHER
...,...,...,...
182231,NCT06875349,Karadeniz Technical University,OTHER
182232,NCT06875557,Fatima Jinnah Women University,OTHER
182233,NCT06875661,University of Novi Sad,OTHER
182234,NCT06875674,University of Palermo,OTHER


# 2) Save Dfs

In [15]:
my_studies.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_studies.pkl")
my_terminations.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_terminations.pkl")  
my_conditions.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_conditions.pkl")  
my_covid.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_covid.pkl")  
my_placebo.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_placebo.pkl")  
my_interventions.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_interventions.pkl")  
my_interventions_types.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_interventions_types.pkl")  
my_soc.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_soc.pkl")  
my_intervention_methods2.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_intervention_methods2.pkl")  
my_intervention_methods.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_intervention_methods.pkl")  
my_adverse.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_adverse.pkl")  
my_adverse_system.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_adverse_system.pkl")  
my_designs.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_designs.pkl")  
my_eligibilities.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_eligibilities.pkl")  
my_outcomes.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_outcomes.pkl")  
my_locations.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_locations.pkl")  
my_documents.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_documents.pkl")  
my_source.to_pickle(r"C:\Users\Eugenia\OneDrive\Documents\THESIS\my_source.pkl")  

NameError: name 'my_soc' is not defined